In [1]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,concatenate, Dropout, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten,Conv2D, Convolution2D, Reshape
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D,Conv2DTranspose
from tensorflow.keras.models import Model

In [2]:
import cv2 
import glob 
import os 
import numpy as np

In [3]:
NO_OF_TRAINING_IMAGES = len(os.listdir('/tensorflow/UBIRS_segnet_data/img_train/'))
NO_OF_VAL_IMAGES = len(os.listdir('/tensorflow/UBIRS_segnet_data/img_test'))


In [4]:
train_in_images_path = '/tensorflow/UBIRS_segnet_data/img_train/'
train_in_labels_path  = '/tensorflow/UBIRS_segnet_data/masks_train/'
test_in_images_path = '/tensorflow/UBIRS_segnet_data/img_test/'
test_in_labels_path = '/tensorflow/UBIRS_segnet_data/masks_test/'

In [5]:
def load_img_data_tensor(path,h,w,c):
    N = len(os.listdir(path))
    data_out= np.zeros((N,h,w,c),'float')
    
    for i in range(0,N):
        
        img_name = os.listdir(path)[i]
        
        img = cv2.imread(path + img_name)
        img = cv2.resize(img,(256,256))
        if(c >1):
            
            img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            data_out[i] = img_rgb
        else:
            
            img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            
            data_out[i] = np.expand_dims(img_gray, axis=2)
        
        #print (i)
    return data_out   
    

In [6]:

img_height = 256
img_width = 256

train_image_orig = load_img_data_tensor(train_in_images_path,img_height,img_width,3)
train_mask_orig = load_img_data_tensor(train_in_labels_path,img_height,img_width,1)
val_image_orig = load_img_data_tensor(test_in_images_path,img_height,img_width,3)
val_mask_orig = load_img_data_tensor(test_in_labels_path,img_height,img_width,1)

In [7]:
X_train = train_image_orig/255.
X_test = val_image_orig/255.
Y_train = train_mask_orig/255.
Y_test = val_mask_orig/255.
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 2100
number of test examples = 150
X_train shape: (2100, 256, 256, 3)
Y_train shape: (2100, 256, 256, 1)
X_test shape: (150, 256, 256, 3)
Y_test shape: (150, 256, 256, 1)


In [8]:
X_test.dtype

dtype('float64')

In [9]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3):
    '''
    Adds 2 convolutional layers with the parameters passed to it

    Args:
    input_tensor (tensor) -- the input tensor
    n_filters (int) -- number of filters
    kernel_size (int) -- kernel size for the convolution

    Returns:
    tensor of output features
    '''
  # first layer
    x = input_tensor
    for i in range(2):
        x = tf.keras.layers.Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
                kernel_initializer = 'he_normal', padding = 'same')(x)
        x = tf.keras.layers.Activation('relu')(x)

    return x


def encoder_block(inputs, n_filters=64, pool_size=(2,2), dropout=0.3):
    '''
    Adds two convolutional blocks and then perform down sampling on output of convolutions.

    Args:
    input_tensor (tensor) -- the input tensor
    n_filters (int) -- number of filters
    kernel_size (int) -- kernel size for the convolution

    Returns:
    f - the output features of the convolution block 
    p - the maxpooled features with dropout
    '''

    f = conv2d_block(inputs, n_filters=n_filters)
    p = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(f)
    p = tf.keras.layers.Dropout(0.3)(p)

    return f, p


In [10]:

def encoder(inputs):
    '''
    This function defines the encoder or downsampling path.

    Args:
    inputs (tensor) -- batch of input images

    Returns:
    p4 - the output maxpooled features of the last encoder block
    (f1, f2, f3, f4) - the output features of all the encoder blocks
    '''
    f1, p1 = encoder_block(inputs, n_filters=64, pool_size=(2,2), dropout=0.3)
    f2, p2 = encoder_block(p1, n_filters=128, pool_size=(2,2), dropout=0.3)
    f3, p3 = encoder_block(p2, n_filters=256, pool_size=(2,2), dropout=0.3)
    f4, p4 = encoder_block(p3, n_filters=512, pool_size=(2,2), dropout=0.3)

    return p4, (f1, f2, f3, f4)

In [11]:
def bottleneck(inputs):
    '''
    This function defines the bottleneck convolutions to extract more features before the upsampling layers.
    '''

    bottle_neck = conv2d_block(inputs, n_filters=1024)

    return bottle_neck

In [12]:
def decoder_block(inputs, conv_output, n_filters=64, kernel_size=3, strides=3, dropout=0.3):
    '''
    defines the one decoder block of the UNet

    Args:
    inputs (tensor) -- batch of input features
    conv_output (tensor) -- features from an encoder block
    n_filters (int) -- number of filters
    kernel_size (int) -- kernel size
    strides (int) -- strides for the deconvolution/upsampling
    padding (string) -- "same" or "valid", tells if shape will be preserved by zero padding

    Returns:
    c (tensor) -- output features of the decoder block
    '''
    u = tf.keras.layers.Conv2DTranspose(n_filters, kernel_size, strides = strides, padding = 'same')(inputs)
    c = tf.keras.layers.concatenate([u, conv_output])
    c = tf.keras.layers.Dropout(dropout)(c)
    c = conv2d_block(c, n_filters, kernel_size=3)

    return c

In [13]:
def decoder(inputs, convs, output_channels):
    '''
    Defines the decoder of the UNet chaining together 4 decoder blocks. 

    Args:
    inputs (tensor) -- batch of input features
    convs (tuple) -- features from the encoder blocks
    output_channels (int) -- number of classes in the label map

    Returns:
    outputs (tensor) -- the pixel wise label map of the image
    '''

    f1, f2, f3, f4 = convs

    c6 = decoder_block(inputs, f4, n_filters=512, kernel_size=(3,3), strides=(2,2), dropout=0.3)
    c7 = decoder_block(c6, f3, n_filters=256, kernel_size=(3,3), strides=(2,2), dropout=0.3)
    c8 = decoder_block(c7, f2, n_filters=128, kernel_size=(3,3), strides=(2,2), dropout=0.3)
    c9 = decoder_block(c8, f1, n_filters=64, kernel_size=(3,3), strides=(2,2), dropout=0.3)

    outputs = tf.keras.layers.Conv2D(output_channels, (1, 1), activation='sigmoid')(c9)

    return outputs

In [14]:
OUTPUT_CHANNELS = 1

def unet():
    '''
    Defines the UNet by connecting the encoder, bottleneck and decoder.
    '''

    # specify the input shape
    inputs = tf.keras.layers.Input(shape=(256,256,3))

    # feed the inputs to the encoder
    encoder_output, convs = encoder(inputs)

    # feed the encoder output to the bottleneck
    bottle_neck = bottleneck(encoder_output)

    # feed the bottleneck and encoder block outputs to the decoder
    # specify the number of classes via the `output_channels` argument
    outputs = decoder(bottle_neck, convs, output_channels=OUTPUT_CHANNELS)

    # create the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model

In [15]:
unetmodel = unet()

# see the resulting model architecture
unetmodel.summary()

2022-02-05 04:43:37.222078: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-05 04:43:37.277400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-05 04:43:37.278667: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-05 04:43:37.281165: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 activation (Activation)        (None, 256, 256, 64  0           ['conv2d[0][0]']                 
                                )                                                             

                                )                                                                 
                                                                                                  
 conv2d_10 (Conv2D)             (None, 32, 32, 512)  4719104     ['dropout_4[0][0]']              
                                                                                                  
 activation_10 (Activation)     (None, 32, 32, 512)  0           ['conv2d_10[0][0]']              
                                                                                                  
 conv2d_11 (Conv2D)             (None, 32, 32, 512)  2359808     ['activation_10[0][0]']          
                                                                                                  
 activation_11 (Activation)     (None, 32, 32, 512)  0           ['conv2d_11[0][0]']              
                                                                                                  
 conv2d_tr

In [16]:
#unetmodel = Unet_model((256,256,3))
optimizer = tf.keras.optimizers.Adam()

In [17]:
unetmodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics = ["accuracy"])

In [ ]:
unetmodel.fit(x = X_train, y = Y_train, epochs = 50, batch_size = 4)

2022-02-05 04:43:39.106077: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1651507200 exceeds 10% of free system memory.
2022-02-05 04:43:40.173250: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 550502400 exceeds 10% of free system memory.
2022-02-05 04:43:40.516699: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1651507200 exceeds 10% of free system memory.
2022-02-05 04:43:41.256708: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 550502400 exceeds 10% of free system memory.


Epoch 1/50


2022-02-05 04:43:43.518769: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2022-02-05 04:43:45.905621: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.17GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-02-05 04:43:46.314710: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.19GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-02-05 04:43:46.373469: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.19GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory

525/525 [==============================] - 197s 363ms/step - loss: 2.4330 - accuracy: 0.9274
Epoch 2/50
525/525 [==============================] - 193s 368ms/step - loss: 0.1569 - accuracy: 0.9283
Epoch 3/50
525/525 [==============================] - 193s 368ms/step - loss: 0.1485 - accuracy: 0.9320
Epoch 4/50
525/525 [==============================] - 193s 368ms/step - loss: 0.1472 - accuracy: 0.9335
Epoch 5/50
525/525 [==============================] - 194s 369ms/step - loss: 0.1680 - accuracy: 0.9315
Epoch 6/50
525/525 [==============================] - 194s 369ms/step - loss: 0.1892 - accuracy: 0.9285
Epoch 7/50
525/525 [==============================] - 194s 370ms/step - loss: 0.1750 - accuracy: 0.9303
Epoch 8/50
525/525 [==============================] - 195s 370ms/step - loss: 0.1705 - accuracy: 0.9308
Epoch 9/50
525/525 [==============================] - 195s 371ms/step - loss: 0.1681 - accuracy: 0.9310
Epoch 10/50
525/525 [==============================] - 195s 370ms/step - lo

In [ ]:
model_save_path = '/tensorflow/unet_model_UBIRIS_1.h5'

unetmodel.save(model_save_path)

In [ ]:
X_test[0:1].shape

In [ ]:
preds = unetmodel.predict(X_test[0:1])

In [ ]:
preds.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
out_image = np.squeeze(preds)


In [ ]:
plt.imshow(out_image,cmap='gray')
plt.show()

In [ ]:
fig=plt.figure(figsize=(8, 8))
columns = 1
rows = 2
fig.add_subplot(rows, columns, 1)
plt.axis("off")
plt.title("input test image")
plt.imshow(X_test[0])
fig.add_subplot(rows, columns, 2)
plt.axis("off")
plt.title("inference_image")
plt.imshow(out_image,cmap='gray')

In [ ]:
print(unetmodel.metrics_names)

In [ ]:
scores_train = unetmodel.evaluate(X_train, Y_train, verbose=0)

In [ ]:
print('train_accuracy ='  ,scores_train[1])

In [ ]:
scores_test = unetmodel.evaluate(X_test, Y_test, verbose=0)

In [ ]:
print('test_accuracy ='  ,scores_test[1])